# Import libraries

In [71]:
import cnn_models
import data_preparation
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from sklearn.svm import SVC
import tensorflow as tf
import data_augmentation
import random

# Fix seed

In [72]:
SEED = 10
random.seed(SEED)
tf.random.set_random_seed(SEED)

# Load recordings
## STANDARD RECORDINGS - No spectrogram normalization

In [4]:
recordings = data_preparation.load_recordings(paths=['recordings', 'output'])

Loading from recordings



Loading from output


Raw recordings have different lengths? Let's check it out:

In [5]:
min_y = min(map(np.shape, recordings))[0]
max_y = max(map(np.shape, recordings))[0]
print(min_y, max_y)

2784 50335


Yes! They vary a lot. For this reason we can add 0s at the beginning and at the end in order to uniform them

**TO DO: Another strategy may be to vary spectrogram params so that spectograms will have the same length**

In [6]:
pad_recordings = data_preparation.pad_zeros(recordings)

pad_zeros >>>
pad_zeros <<<


What is the range now?

In [7]:
min_y = min(map(np.shape, pad_recordings))[0]
max_y = max(map(np.shape, pad_recordings))[0]
print(min_y, max_y)

50335 50335


We can now compute spectograms:

In [8]:
spects = [data_preparation.compute_spectrogram(x) for x in pad_recordings]
spects = np.array(spects)

The procedure worked as expected! we can now move on to the prediction task

In [9]:
labels_speakers = data_preparation.load_labels(paths=['recordings', 'output'], label_type="speakers")
labels_digits = data_preparation.load_labels(paths=['recordings', 'output'])

Normalize spectrograms

In [10]:
norm_spects = [data_preparation.compute_spectrogram(x, normalize=True) for x in pad_recordings]
norm_spects = np.array(norm_spects)

## Augmentation

In [73]:
%%time
X_train_digit, y_train_digit, X_test_digit, y_test_digit = data_preparation.prepare_augmented_recordings(audio_dirs= ['output', 'recordings'],
                             y_type= ['digit', 'digit'],
                             n_category_test=15,
                             include_pitch=True)

conversion_done!
compute_spectrograms >>>
pad_zeros >>>
pad_zeros <<<
pad_zeros >>>
pad_zeros <<<
Padding done
compute_spectrograms <<<
CPU times: user 7min 3s, sys: 23 s, total: 7min 26s
Wall time: 5min 31s


In [12]:
print("Lengths : {}, {}, {}, {}".format(len(X_train_digit),
                                                 len(y_train_digit),
                                                 len(X_test_digit),
                                                 len(y_test_digit),))

Lengths : 23100, 23100, 300, 300


In [13]:
%%time
X_train_speaker, y_train_speaker, X_test_speaker, y_test_speaker = data_preparation.prepare_augmented_recordings(audio_dirs= ['output', 'recordings'],
                             y_type= ['speakers_us', 'speakers_default'],
                             n_category_test=30,
                             include_pitch=True)

conversion_done!
compute_spectrograms >>>
pad_zeros >>>
pad_zeros <<<
pad_zeros >>>
pad_zeros <<<
Padding done
compute_spectrograms <<<
CPU times: user 6min 52s, sys: 21 s, total: 7min 13s
Wall time: 5min 26s


In [14]:
print("Lengths : {}, {}, {}, {}".format(len(X_train_speaker),
                                                 len(y_train_speaker),
                                                 len(X_test_speaker),
                                                 len(y_test_speaker)))

Lengths : 23760, 23760, 240, 240


# Standard recordings
## Numbers

Split data in train and test

In [15]:
X_train, X_test, y_train, y_test = data_preparation.split_train_test_baseline_spectrograms(spects, labels_digits)

In [16]:
clf1 = SVC(kernel='rbf', class_weight='balanced', gamma="auto")

In [17]:
%%time
clf1 = clf1.fit(X_train, y_train)

CPU times: user 3min 56s, sys: 1.77 s, total: 3min 58s
Wall time: 4min 6s


In [18]:
%%time
y_pred = clf1.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.27      0.42        41
           1       0.51      0.50      0.51        40
           2       0.81      0.26      0.39        50
           3       0.52      0.27      0.35        52
           4       0.27      0.48      0.35        48
           5       0.94      0.36      0.52        45
           6       0.16      0.55      0.25        62
           7       0.72      0.39      0.51        46
           8       0.85      0.35      0.49        49
           9       0.86      0.51      0.64        47

    accuracy                           0.40       480
   macro avg       0.66      0.39      0.44       480
weighted avg       0.64      0.40      0.43       480

CPU times: user 37.9 s, sys: 260 ms, total: 38.1 s
Wall time: 38.4 s


### Normalize spectrograms

In [19]:
X_train, X_test, y_train, y_test = data_preparation.split_train_test_baseline_spectrograms(norm_spects, labels_digits)

In [20]:
%%time
clf1 = SVC(kernel='rbf', class_weight='balanced', gamma="auto")
clf1 = clf1.fit(X_train, y_train)

CPU times: user 3min, sys: 2.48 s, total: 3min 2s
Wall time: 3min 22s


In [21]:
%%time
y_pred = clf1.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.83      0.85        41
           1       0.79      0.82      0.80        40
           2       0.56      0.90      0.69        50
           3       0.77      0.44      0.56        52
           4       0.93      0.85      0.89        48
           5       0.87      0.73      0.80        45
           6       0.61      0.74      0.67        62
           7       0.70      0.85      0.76        46
           8       0.82      0.73      0.77        49
           9       0.97      0.64      0.77        47

    accuracy                           0.75       480
   macro avg       0.79      0.75      0.76       480
weighted avg       0.78      0.75      0.75       480

CPU times: user 36.7 s, sys: 337 ms, total: 37 s
Wall time: 39.2 s


### CNNs

#### Normalized spectrograms

In [22]:
X_train, X_test, y_train, y_test, input_shape = data_preparation.split_train_test_nn(norm_spects, labels_digits)

In [23]:
model = cnn_models.paper_architecture(10, input_shape=input_shape)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 63, 156, 32)       544       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 30, 77, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 37, 64)        32832     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 17, 64)         0         
_________________________________________________________________
flatten (Flatten)            (None, 6528)              0         
_________________________________________________________________
dense (Dense)        

In [24]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', restore_best_weights=True, patience=3)

In [25]:
%%time
model.fit(X_train, y_train,
          batch_size=32,
          epochs=10,
          verbose=1,
          callbacks=[callback],
          validation_data=(X_test, y_test))

Train on 1920 samples, validate on 480 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
1920/1920 [==============================] - 24s 12ms/sample - loss: 2.2689 - acc: 0.1703 - val_loss: 2.2025 - val_acc: 0.2646
Epoch 2/10
1920/1920 [==============================] - 26s 13ms/sample - loss: 2.1098 - acc: 0.2615 - val_loss: 1.9102 - val_acc: 0.4333
Epoch 3/10
1920/1920 [==============================] - 23s 12ms/sample - loss: 1.8666 - acc: 0.3516 - val_loss: 1.6058 - val_acc: 0.5292
Epoch 4/10
1920/1920 [==============================] - 20s 11ms/sample - loss: 1.6297 - acc: 0.4187 - val_loss: 1.3857 - val_acc: 0.5188
Epoch 5/10
1920/1920 [==============================] - 22s 12ms/sample - loss: 1.4626 - acc: 0.4927 - val_loss: 1.1161 - val_acc: 0.6542
Epoch 6/10
1920/1920 [==============================] - 28s 14ms/sample - loss: 1.3157 - acc: 0.5365 - val_loss: 1.0770 - val_acc: 0.6146
Epoch 7/10
1920/1920 [==============================] - 26s 13ms/sample - los

In [26]:
Y_test = np.argmax(y_test, axis=1) # Convert one-hot to index
y_pred = model.predict_classes(X_test)
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.95      0.94        41
           1       0.94      0.75      0.83        40
           2       0.89      0.82      0.85        50
           3       0.77      0.63      0.69        52
           4       0.94      0.92      0.93        48
           5       0.86      0.71      0.78        45
           6       0.87      0.65      0.74        62
           7       0.63      0.96      0.76        46
           8       0.71      0.76      0.73        49
           9       0.68      0.94      0.79        47

    accuracy                           0.80       480
   macro avg       0.82      0.81      0.80       480
weighted avg       0.82      0.80      0.80       480



#### Standard spectrogram

In [27]:
X_train, X_test, y_train, y_test, input_shape = data_preparation.split_train_test_nn(spects, labels_digits)

In [28]:
model = cnn_models.paper_architecture(10, input_shape)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 63, 156, 32)       544       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 77, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 37, 64)        32832     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 17, 64)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6528)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               652900    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
__________

In [29]:
%%time
model.fit(X_train, y_train,
          batch_size=32,
          epochs=10,
          verbose=1,
          callbacks=[callback],
          validation_data=(X_test, y_test))

Train on 1920 samples, validate on 480 samples
Epoch 1/10
1920/1920 [==============================] - 23s 12ms/sample - loss: 2.3112 - acc: 0.1542 - val_loss: 2.1489 - val_acc: 0.2792
Epoch 2/10
1920/1920 [==============================] - 20s 10ms/sample - loss: 2.1599 - acc: 0.2438 - val_loss: 2.0400 - val_acc: 0.3562
Epoch 3/10
1920/1920 [==============================] - 20s 10ms/sample - loss: 2.1303 - acc: 0.2620 - val_loss: 2.0265 - val_acc: 0.3042
Epoch 4/10
1920/1920 [==============================] - 20s 11ms/sample - loss: 2.0835 - acc: 0.2646 - val_loss: 2.0087 - val_acc: 0.3042
Epoch 5/10
1920/1920 [==============================] - 21s 11ms/sample - loss: 2.0202 - acc: 0.3010 - val_loss: 1.9423 - val_acc: 0.3250
Epoch 6/10
1920/1920 [==============================] - 21s 11ms/sample - loss: 1.9777 - acc: 0.3255 - val_loss: 1.8864 - val_acc: 0.3521
Epoch 7/10
1920/1920 [==============================] - 20s 10ms/sample - loss: 1.9308 - acc: 0.3328 - val_loss: 1.8352 - val

In [30]:
Y_test = np.argmax(y_test, axis=1) # Convert one-hot to index
y_pred = model.predict_classes(X_test)
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.34      0.47        41
           1       0.40      0.53      0.46        40
           2       0.41      0.78      0.54        50
           3       0.38      0.29      0.33        52
           4       0.92      0.25      0.39        48
           5       0.70      0.31      0.43        45
           6       0.64      0.37      0.47        62
           7       0.71      0.37      0.49        46
           8       0.55      0.24      0.34        49
           9       0.28      0.96      0.43        47

    accuracy                           0.44       480
   macro avg       0.58      0.44      0.43       480
weighted avg       0.58      0.44      0.43       480



From what we can see normalising spectrograms is the way to go. Let's use it by default

## Speakers
### SVD

In [31]:
X_train, X_test, y_train, y_test = data_preparation.split_train_test_baseline_spectrograms(norm_spects, labels_speakers)

In [32]:
%%time
clf1 = SVC(kernel='rbf', class_weight='balanced', gamma="auto")
clf1 = clf1.fit(X_train, y_train)

CPU times: user 2min 1s, sys: 1.59 s, total: 2min 3s
Wall time: 2min 10s


In [33]:
%%time
y_pred = clf1.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         ale       1.00      0.87      0.93        15
      alinda       0.93      1.00      0.97        14
        gian       0.87      1.00      0.93        20
     jackson       0.98      0.95      0.97       109
      khaled       0.78      1.00      0.88        14
     nicolas       0.99      0.95      0.97        91
        theo       0.75      0.87      0.81       106
    yweweler       0.89      0.77      0.82       111

    accuracy                           0.89       480
   macro avg       0.90      0.92      0.91       480
weighted avg       0.90      0.89      0.89       480

CPU times: user 33.5 s, sys: 256 ms, total: 33.8 s
Wall time: 34 s


### CNN

For neural networks it is not possible to pass the labels as-is: we need to transform them in numbers. The safest way is through one-hot encoding

In [34]:
y, target_names = data_preparation.transform_categorical_y(labels_speakers)

In [35]:
X_train, X_test, y_train, y_test, input_shape = data_preparation.split_train_test_nn(norm_spects, y, number_mode=False)

In [36]:
model = cnn_models.paper_architecture(8, input_shape=input_shape)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 63, 156, 32)       544       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 30, 77, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 37, 64)        32832     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 6, 17, 64)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 6528)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 80)                522320    
_________________________________________________________________
dropout_2 (Dropout)          (None, 80)                0         
__________

In [37]:
%%time
model.fit(X_train, y_train,
          batch_size=32,
          epochs=10,
          verbose=1,
          callbacks=[callback],
          validation_data=(X_test, y_test))

Train on 1920 samples, validate on 480 samples
Epoch 1/10
1920/1920 [==============================] - 22s 12ms/sample - loss: 1.9018 - acc: 0.2453 - val_loss: 1.6902 - val_acc: 0.3854
Epoch 2/10
1920/1920 [==============================] - 19s 10ms/sample - loss: 1.7141 - acc: 0.3979 - val_loss: 1.5000 - val_acc: 0.4229
Epoch 3/10
1920/1920 [==============================] - 20s 10ms/sample - loss: 1.5235 - acc: 0.5109 - val_loss: 1.1671 - val_acc: 0.7000
Epoch 4/10
1920/1920 [==============================] - 19s 10ms/sample - loss: 1.3174 - acc: 0.5771 - val_loss: 0.9580 - val_acc: 0.7042
Epoch 5/10
1920/1920 [==============================] - 21s 11ms/sample - loss: 1.1541 - acc: 0.6099 - val_loss: 0.7775 - val_acc: 0.7500
Epoch 6/10
1920/1920 [==============================] - 19s 10ms/sample - loss: 1.0102 - acc: 0.6578 - val_loss: 0.6683 - val_acc: 0.7708
Epoch 7/10
1920/1920 [==============================] - 19s 10ms/sample - loss: 0.9052 - acc: 0.6969 - val_loss: 0.6118 - val

In [38]:
Y_test = np.argmax(y_test, axis=1) # Convert one-hot to index
y_pred = model.predict_classes(X_test)
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.73      0.73      0.73        15
           1       0.30      0.50      0.38        14
           2       0.71      0.25      0.37        20
           3       0.99      0.72      0.84       109
           4       0.75      0.43      0.55        14
           5       0.79      0.97      0.87        91
           6       0.81      0.87      0.84       106
           7       0.83      0.91      0.87       111

    accuracy                           0.81       480
   macro avg       0.74      0.67      0.68       480
weighted avg       0.83      0.81      0.81       480



#### Paper - batch_normalisation=True

In [39]:
model = cnn_models.paper_architecture(8, input_shape, batch_normalisation=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 63, 156, 32)       544       
_________________________________________________________________
batch_normalization_v1 (Batc (None, 63, 156, 32)       128       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 30, 77, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 14, 37, 64)        32832     
_________________________________________________________________
batch_normalization_v1_1 (Ba (None, 14, 37, 64)        256       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 6, 17, 64)         0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 6528)              0         
__________

In [40]:
%%time
model.fit(X_train, y_train,
          batch_size=32,
          epochs=10,
          verbose=1,
          callbacks=[callback],
          validation_data=(X_test, y_test))

Train on 1920 samples, validate on 480 samples
Epoch 1/10
1920/1920 [==============================] - 48s 25ms/sample - loss: 1.1804 - acc: 0.6203 - val_loss: 1.7356 - val_acc: 0.4292
Epoch 2/10
1920/1920 [==============================] - 50s 26ms/sample - loss: 0.6233 - acc: 0.8229 - val_loss: 1.6220 - val_acc: 0.4333
Epoch 3/10
1920/1920 [==============================] - 48s 25ms/sample - loss: 0.4460 - acc: 0.8760 - val_loss: 1.6398 - val_acc: 0.4271
Epoch 4/10
1920/1920 [==============================] - 49s 26ms/sample - loss: 0.3360 - acc: 0.9141 - val_loss: 1.4018 - val_acc: 0.4479
Epoch 5/10
1920/1920 [==============================] - 59s 31ms/sample - loss: 0.2615 - acc: 0.9427 - val_loss: 1.2584 - val_acc: 0.4604
Epoch 6/10
1920/1920 [==============================] - 53s 28ms/sample - loss: 0.2195 - acc: 0.9510 - val_loss: 1.0832 - val_acc: 0.5292
Epoch 7/10
1920/1920 [==============================] - 53s 28ms/sample - loss: 0.1957 - acc: 0.9563 - val_loss: 0.9620 - val

In [41]:
Y_test = np.argmax(y_test, axis=1) # Convert one-hot to index
y_pred = model.predict_classes(X_test)
print(classification_report(Y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

         ale       1.00      0.93      0.97        15
      alinda       1.00      0.36      0.53        14
        gian       0.70      0.80      0.74        20
     jackson       0.83      1.00      0.91       109
      khaled       1.00      0.86      0.92        14
     nicolas       1.00      0.88      0.94        91
        theo       0.99      0.92      0.95       106
        theo       0.95      1.00      0.97       111

    accuracy                           0.93       480
   macro avg       0.93      0.84      0.87       480
weighted avg       0.94      0.93      0.92       480



# Data augmentation
## Speaker

In [42]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
y_train_speaker = enc.fit_transform(np.array(y_train_speaker).reshape(-1, 1)).toarray()
y_test_speaker = enc.transform(np.array(y_test_speaker).reshape(-1, 1)).toarray()
label_0 = enc.inverse_transform(np.array([1, 0, 0, 0, 0, 0, 0, 0]).reshape(1, -1))[0][0]
label_1 = enc.inverse_transform(np.array([0, 1, 0, 0, 0, 0, 0, 0]).reshape(1, -1))[0][0]
label_2 = enc.inverse_transform(np.array([0, 0, 1, 0, 0, 0, 0, 0]).reshape(1, -1))[0][0]
label_3 = enc.inverse_transform(np.array([0, 0, 0, 1, 0, 0, 0, 0]).reshape(1, -1))[0][0]
label_4 = enc.inverse_transform(np.array([0, 0, 0, 0, 1, 0, 0, 0]).reshape(1, -1))[0][0]
label_5 = enc.inverse_transform(np.array([0, 0, 0, 0, 0, 1, 0, 0]).reshape(1, -1))[0][0]
label_6 = enc.inverse_transform(np.array([0, 0, 0, 0, 0, 0, 1, 0]).reshape(1, -1))[0][0]
label_7 = enc.inverse_transform(np.array([0, 0, 0, 0, 0, 0, 0, 1]).reshape(1, -1))[0][0]
target_names = [label_0, label_1, label_2, label_3, label_4, label_5, label_6, label_7]

In [43]:
y_test_speaker

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [44]:
X_train_speaker = np.array(X_train_speaker)
X_test_speaker = np.array(X_test_speaker)

In [45]:
X_train_speaker = X_train_speaker.reshape(X_train_speaker.shape[0],
                                          X_train_speaker.shape[1],
                                          X_train_speaker.shape[2],
                                          1)
X_test_speaker = X_test_speaker.reshape(X_test_speaker.shape[0], X_test_speaker.shape[1], X_test_speaker.shape[2], 1)

In [46]:
input_shape = (X_train_speaker.shape[1], X_train_speaker.shape[2], 1)

In [47]:
model = cnn_models.paper_architecture(8, input_shape=input_shape)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 63, 56, 32)        544       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 30, 27, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 14, 12, 64)        32832     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 6, 5, 64)          0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 80)                153680    
_________________________________________________________________
dropout_4 (Dropout)          (None, 80)                0         
__________

In [48]:
%%time
model.fit(X_train_speaker, y_train_speaker,
          batch_size=32,
          epochs=10,
          verbose=1,
          callbacks=[callback],
          validation_data=(X_test_speaker, y_test_speaker))

Train on 23760 samples, validate on 240 samples
Epoch 1/10
23760/23760 [==============================] - 82s 3ms/sample - loss: 1.3939 - acc: 0.4953 - val_loss: 1.5296 - val_acc: 0.4542
Epoch 2/10
23760/23760 [==============================] - 78s 3ms/sample - loss: 0.9199 - acc: 0.6582 - val_loss: 1.1219 - val_acc: 0.5667
Epoch 3/10
23760/23760 [==============================] - 79s 3ms/sample - loss: 0.7097 - acc: 0.7293 - val_loss: 0.7211 - val_acc: 0.7042
Epoch 4/10
23760/23760 [==============================] - 79s 3ms/sample - loss: 0.6206 - acc: 0.7652 - val_loss: 0.6700 - val_acc: 0.7583
Epoch 5/10
23760/23760 [==============================] - 78s 3ms/sample - loss: 0.5283 - acc: 0.7982 - val_loss: 0.4975 - val_acc: 0.8083
Epoch 6/10
23760/23760 [==============================] - 93s 4ms/sample - loss: 0.4776 - acc: 0.8191 - val_loss: 0.6120 - val_acc: 0.7250
Epoch 7/10
23760/23760 [==============================] - 138s 6ms/sample - loss: 0.4243 - acc: 0.8393 - val_loss: 0.3

In [49]:
Y_test = np.argmax(y_test_speaker, axis=1) # Convert one-hot to index
y_pred = model.predict_classes(X_test_speaker)
print(classification_report(Y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

         ale       0.93      0.93      0.93        30
      alinda       0.93      0.93      0.93        30
        gian       1.00      0.97      0.98        30
     jackson       0.83      1.00      0.91        30
      khaled       1.00      0.77      0.87        30
     nicolas       0.97      1.00      0.98        30
        theo       0.97      0.93      0.95        30
    yweweler       0.94      1.00      0.97        30

    accuracy                           0.94       240
   macro avg       0.95      0.94      0.94       240
weighted avg       0.95      0.94      0.94       240



### Batch_normalization = True

In [50]:
model = cnn_models.paper_architecture(8, input_shape=input_shape, batch_normalisation=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 63, 56, 32)        544       
_________________________________________________________________
batch_normalization_v1_4 (Ba (None, 63, 56, 32)        128       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 30, 27, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 14, 12, 64)        32832     
_________________________________________________________________
batch_normalization_v1_5 (Ba (None, 14, 12, 64)        256       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 6, 5, 64)          0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 1920)              0         
__________

In [51]:
%%time
model.fit(X_train_speaker, y_train_speaker,
          batch_size=32,
          epochs=10,
          verbose=1,
          callbacks=[callback],
          validation_data=(X_test_speaker, y_test_speaker))

Train on 23760 samples, validate on 240 samples
Epoch 1/10
23760/23760 [==============================] - 251s 11ms/sample - loss: 0.9276 - acc: 0.6698 - val_loss: 0.6872 - val_acc: 0.7625
Epoch 2/10
23760/23760 [==============================] - 225s 9ms/sample - loss: 0.5510 - acc: 0.7910 - val_loss: 0.2836 - val_acc: 0.9292
Epoch 3/10
23760/23760 [==============================] - 226s 10ms/sample - loss: 0.4298 - acc: 0.8372 - val_loss: 0.2189 - val_acc: 0.9292
Epoch 4/10
23760/23760 [==============================] - 230s 10ms/sample - loss: 0.3748 - acc: 0.8571 - val_loss: 0.2276 - val_acc: 0.9167
Epoch 5/10
23760/23760 [==============================] - 303s 13ms/sample - loss: 0.3342 - acc: 0.8735 - val_loss: 0.1035 - val_acc: 0.9708
Epoch 6/10
23760/23760 [==============================] - 341s 14ms/sample - loss: 0.2997 - acc: 0.8879 - val_loss: 0.2797 - val_acc: 0.8833
Epoch 7/10
23760/23760 [==============================] - 360s 15ms/sample - loss: 0.2897 - acc: 0.8893 - v

In [52]:
Y_test = np.argmax(y_test_speaker, axis=1) # Convert one-hot to index
y_pred = model.predict_classes(X_test_speaker)
print(classification_report(Y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

         ale       1.00      0.97      0.98        30
      alinda       1.00      1.00      1.00        30
        gian       1.00      0.97      0.98        30
     jackson       0.94      1.00      0.97        30
      khaled       1.00      0.93      0.97        30
     nicolas       0.97      1.00      0.98        30
        theo       0.97      1.00      0.98        30
    yweweler       1.00      1.00      1.00        30

    accuracy                           0.98       240
   macro avg       0.98      0.98      0.98       240
weighted avg       0.98      0.98      0.98       240



### Different architecture
Let's change a bit the architecture and see if we can improve scores:

In [53]:
model = cnn_models.custom_cnn(8, input_shape=input_shape)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 63, 56, 32)        544       
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 30, 27, 64)        32832     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 14, 12, 64)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 10752)             0         
_________________________________________________________________
dense_18 (Dense)             (None, 128)               1376384   
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 8)                 1032      
Total para

In [54]:
%%time
model.fit(X_train_speaker, y_train_speaker,
          batch_size=32,
          epochs=10,
          verbose=1,
          callbacks=[callback],
          validation_data=(X_test_speaker, y_test_speaker))

Train on 23760 samples, validate on 240 samples
Epoch 1/10
23760/23760 [==============================] - 212s 9ms/sample - loss: 1.0306 - acc: 0.6256 - val_loss: 0.6918 - val_acc: 0.7375
Epoch 2/10
23760/23760 [==============================] - 179s 8ms/sample - loss: 0.5285 - acc: 0.7957 - val_loss: 0.3108 - val_acc: 0.9000
Epoch 3/10
23760/23760 [==============================] - 175s 7ms/sample - loss: 0.3513 - acc: 0.8658 - val_loss: 0.1635 - val_acc: 0.9500
Epoch 4/10
23760/23760 [==============================] - 176s 7ms/sample - loss: 0.2707 - acc: 0.8956 - val_loss: 0.1519 - val_acc: 0.9542
Epoch 5/10
23760/23760 [==============================] - 178s 8ms/sample - loss: 0.2200 - acc: 0.9146 - val_loss: 0.0805 - val_acc: 0.9833
Epoch 6/10
23760/23760 [==============================] - 179s 8ms/sample - loss: 0.1928 - acc: 0.9231 - val_loss: 0.1015 - val_acc: 0.9792
Epoch 7/10
23760/23760 [==============================] - 177s 7ms/sample - loss: 0.1699 - acc: 0.9328 - val_los

In [55]:
Y_test = np.argmax(y_test_speaker, axis=1) # Convert one-hot to index
y_pred = model.predict_classes(X_test_speaker)
print(classification_report(Y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

         ale       1.00      0.97      0.98        30
      alinda       1.00      1.00      1.00        30
        gian       0.97      1.00      0.98        30
     jackson       1.00      1.00      1.00        30
      khaled       1.00      0.97      0.98        30
     nicolas       1.00      1.00      1.00        30
        theo       0.94      0.97      0.95        30
    yweweler       0.97      0.97      0.97        30

    accuracy                           0.98       240
   macro avg       0.98      0.98      0.98       240
weighted avg       0.98      0.98      0.98       240



### Digits

In [74]:
X_train_digit_nn = np.array(X_train_digit)
X_test_digit_nn = np.array(X_test_digit)
X_train_digit_nn = X_train_digit_nn.reshape(X_train_digit_nn.shape[0], X_train_digit_nn.shape[1], X_train_digit_nn.shape[2], 1)
X_test_digit_nn = X_test_digit_nn.reshape(X_test_digit_nn.shape[0], X_test_digit_nn.shape[1], X_test_digit_nn.shape[2], 1)
y_train_digit_nn = tf.keras.utils.to_categorical(y_train_digit, 10)
y_test_digit_nn = tf.keras.utils.to_categorical(y_test_digit, 10)

In [85]:
input_shape = (X_train_digit_nn.shape[1], X_train_digit_nn.shape[2], 1)

#### Paper

In [86]:
model = cnn_models.paper_architecture(10, input_shape=input_shape, batch_normalisation=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 63, 56, 32)        544       
_________________________________________________________________
batch_normalization_v1_24 (B (None, 63, 56, 32)        128       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 30, 27, 32)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 14, 12, 64)        32832     
_________________________________________________________________
batch_normalization_v1_25 (B (None, 14, 12, 64)        256       
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 6, 5, 64)          0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 1920)              0         
__________

In [92]:
%%time
model.fit(X_train_digit_nn, y_train_digit_nn,
          batch_size=32,
          epochs=10,
          verbose=1,
          callbacks=[callback],
          validation_data=(X_test_digit_nn, y_test_digit_nn))

Train on 23100 samples, validate on 300 samples
Epoch 1/10
23100/23100 [==============================] - 216s 9ms/sample - loss: 1.5178 - acc: 0.4597 - val_loss: 0.9186 - val_acc: 0.6867
Epoch 2/10
23100/23100 [==============================] - 206s 9ms/sample - loss: 1.0116 - acc: 0.6526 - val_loss: 0.5175 - val_acc: 0.8400
Epoch 3/10
23100/23100 [==============================] - 212s 9ms/sample - loss: 0.8419 - acc: 0.7142 - val_loss: 0.3905 - val_acc: 0.9033
Epoch 4/10
23100/23100 [==============================] - 208s 9ms/sample - loss: 0.7701 - acc: 0.7374 - val_loss: 0.3749 - val_acc: 0.8800
Epoch 5/10
23100/23100 [==============================] - 221s 10ms/sample - loss: 0.6947 - acc: 0.7643 - val_loss: 0.3934 - val_acc: 0.8833
Epoch 6/10
23100/23100 [==============================] - 215s 9ms/sample - loss: 0.6530 - acc: 0.7807 - val_loss: 0.2742 - val_acc: 0.9167
Epoch 7/10
23100/23100 [==============================] - 209s 9ms/sample - loss: 0.6182 - acc: 0.7917 - val_lo

In [96]:
Y_test = np.argmax(y_test_digit_nn, axis=1) # Convert one-hot to index
y_pred = model.predict_classes(X_test_digit_nn)
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98        30
           1       0.96      0.87      0.91        30
           2       0.88      0.97      0.92        30
           3       0.90      0.90      0.90        30
           4       0.88      1.00      0.94        30
           5       0.90      0.87      0.88        30
           6       0.96      0.87      0.91        30
           7       0.94      0.97      0.95        30
           8       0.90      0.90      0.90        30
           9       0.96      0.90      0.93        30

    accuracy                           0.92       300
   macro avg       0.93      0.92      0.92       300
weighted avg       0.93      0.92      0.92       300



#### Custom

In [98]:
model = cnn_models.custom_cnn(10, input_shape=input_shape)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 63, 56, 32)        544       
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 30, 27, 64)        32832     
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 14, 12, 64)        0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 10752)             0         
_________________________________________________________________
dense_35 (Dense)             (None, 128)               1376384   
_________________________________________________________________
dropout_12 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_36 (Dense)             (None, 10)                1290      
Total para

In [99]:
%%time
model.fit(X_train_digit_nn, y_train_digit_nn,
          batch_size=32,
          epochs=10,
          verbose=1,
          callbacks=[callback],
          validation_data=(X_test_digit_nn, y_test_digit_nn))

Train on 23100 samples, validate on 300 samples
Epoch 1/10
23100/23100 [==============================] - 278s 12ms/sample - loss: 1.5071 - acc: 0.4624 - val_loss: 0.7149 - val_acc: 0.7833
Epoch 2/10
23100/23100 [==============================] - 208s 9ms/sample - loss: 0.9832 - acc: 0.6505 - val_loss: 0.4521 - val_acc: 0.8567
Epoch 3/10
23100/23100 [==============================] - 222s 10ms/sample - loss: 0.7937 - acc: 0.7180 - val_loss: 0.3445 - val_acc: 0.8933
Epoch 4/10
23100/23100 [==============================] - 225s 10ms/sample - loss: 0.6822 - acc: 0.7599 - val_loss: 0.2504 - val_acc: 0.9200
Epoch 5/10
23100/23100 [==============================] - 246s 11ms/sample - loss: 0.6088 - acc: 0.7839 - val_loss: 0.2348 - val_acc: 0.9300
Epoch 6/10
23100/23100 [==============================] - 216s 9ms/sample - loss: 0.5535 - acc: 0.8026 - val_loss: 0.3203 - val_acc: 0.9067
Epoch 7/10
23100/23100 [==============================] - 196s 8ms/sample - loss: 0.5150 - acc: 0.8172 - val

In [100]:
Y_test = np.argmax(y_test_digit_nn, axis=1) # Convert one-hot to index
y_pred = model.predict_classes(X_test_digit_nn)
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.93      0.97        30
           1       1.00      0.93      0.97        30
           2       0.86      1.00      0.92        30
           3       0.88      1.00      0.94        30
           4       1.00      1.00      1.00        30
           5       0.94      0.97      0.95        30
           6       0.96      0.87      0.91        30
           7       0.91      1.00      0.95        30
           8       0.96      0.87      0.91        30
           9       1.00      0.90      0.95        30

    accuracy                           0.95       300
   macro avg       0.95      0.95      0.95       300
weighted avg       0.95      0.95      0.95       300



# Test model 

In [56]:
import sounddevice as sd
import subprocess

import time
import librosa

import IPython.display as ipd

import os
from scipy.io import wavfile as wav

In [ ]:
def pad_zeros_single_rec(rec, max_y):
    rec = np.array(rec)
    diff_in_y = max_y - rec.shape[0]
    if diff_in_y > 0:
        half_diff = int(diff_in_y/2)
        remaining_diff = diff_in_y-half_diff
        v = np.pad(rec, (half_diff, remaining_diff), 'constant', constant_values=0)
        return v
    else:
        return rec

In [ ]:
def create_recording(duration, rec_rate, name = "test.wav", output_dir = "test/"):
    print("Ready in 3...", end = "")
    time.sleep(1)
    print("2...", end = "")
    time.sleep(1)
    print("1...")
    time.sleep(1)
    print("Go.")
    rec = sd.rec(int(duration * rec_rate), samplerate=rec_rate, channels=1, blocking=True)
    print("Playing the recording.")
    sd.play(rec, rec_rate)

    # after hearing the recording, decide whether to record it again or continue to next number
    # if you type anything, record again
    # if you press enter, save current recording & go to next number
    ok = input("OK?")
    if ok == "":
        librosa.output.write_wav(output_dir+name, rec, rec_rate)
        return rec
    ipd.clear_output(wait=True)
    create_recording(duration, rec_rate)

In [ ]:
def trim_audio(file, input_dir="test/", output_dir="test/", db=-48):

    if not os.path.isdir(input_dir):
        print(f"There should be an input \"{input_dir}\" directory.")
        sys.exit(0)
    
    # create output directory if not there yet
    if not os.path.isdir(output_dir):
        os.makedirs(output_dir)
        
    temp1 = output_dir+"temp1.wav"
    temp2 = output_dir+"temp2.wav"
    temp3 = output_dir+"temp3.wav"
 
    subprocess.run(["ffmpeg", "-y", "-i", input_dir+file, "-af", f"silenceremove=1:0:{db}dB", temp1])
    subprocess.run(["ffmpeg", "-y", "-i", temp1, "-af", "areverse", temp2])
    subprocess.run(["ffmpeg", "-y", "-i", temp2, "-af", f"silenceremove=1:0.1:{db}dB", temp3])
    subprocess.run(["ffmpeg", "-y", "-i", temp3, "-af", "areverse", output_dir+file])
    
    os.remove(temp1)
    os.remove(temp2)
    os.remove(temp3)

In [ ]:
def test_NN(nn, max_y, target_names, answer = None, duration=2, rec_rate=8000, directory = "test/", filename = "test.wav"):
    create_recording(duration, rec_rate, filename, directory)   
    ipd.clear_output()
    trim_audio(filename, directory, directory)
    # _, rec = wav.read(directory + "/" + filename)
    rec, _ = librosa.core.load(directory + "/" + filename, sr = rec_rate)
    rec = pad_zeros_single_rec(rec, max_y)
    # sd.play(rec, rec_rate)
    rec = data_preparation.compute_spectrogram(rec, normalize=True)
    rec = rec[np.newaxis,:,:,np.newaxis]
    preds = nn.predict_classes(rec)
    print("Model prediction: {}".format(target_names[preds[0]]))
    if answer is not None:
        print(f"Correct answer {answer}")
    return preds

In [ ]:
max_y = len(data_augm_pad_recordings[1])

In [ ]:
pred = test_NN(model, max_y, target_names, answer = "gian")

# TO DO:
- [x] Set random seed
- [x] Use only original recordings in test set of augmented scenario
- Use proper validation set (optional: also crossvalidation) for picking best models and params
- Data augmentation also for digit recognition